## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-25-20-51-57 +0000,nypost,Council gearing up to nix Mayor Adams’ Christm...,https://nypost.com/2025/12/25/us-news/council-...
1,2025-12-25-20-45-15 +0000,nypost,Ukraine delivers humiliating Christmas Day blo...,https://nypost.com/2025/12/25/world-news/ukrai...
2,2025-12-25-20-18-21 +0000,nypost,Martin Scorsese recalls tender Rob Reiner scen...,https://nypost.com/2025/12/25/us-news/martin-s...
3,2025-12-25-20-16-11 +0000,startribune,Live: Max Brosmer leads the Vikings against th...,https://www.startribune.com/minnesota-vikings-...
4,2025-12-25-20-13-57 +0000,nyt,Christmas Around the World in Photos,https://www.nytimes.com/2025/12/25/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,christmas,40
136,trump,25
81,year,10
127,new,8
43,world,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
125,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...,93
82,2025-12-25-12-09-11 +0000,nyt,Kennedy Center’s Christmas Eve Jazz Show Cance...,https://www.nytimes.com/2025/12/24/us/kennedy-...,90
151,2025-12-25-00-41-15 +0000,bbc,Where is Santa now? Trump answers kids' Christ...,https://www.bbc.com/news/videos/cqxqvd0z5gno?a...,78
71,2025-12-25-15-07-19 +0000,nypost,Trump wishes all — including ‘Radical Left Scu...,https://nypost.com/2025/12/25/us-news/trump-wi...,78
17,2025-12-25-19-28-21 +0000,nypost,New Yorkers shelled out more for Christmas gif...,https://nypost.com/2025/12/25/lifestyle/parent...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
125,93,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...
9,43,2025-12-25-19-55-37 +0000,nypost,3-year-old boy orphaned when wrong-way drunken...,https://nypost.com/2025/12/25/us-news/texas-bo...
58,37,2025-12-25-16-21-18 +0000,bbc,Pope Leo urges 'courage' to end Ukraine war in...,https://www.bbc.com/news/articles/cm21728nwxlo...
165,34,2025-12-24-23-12-53 +0000,wapo,Trump-backed conservative candidate wins Hondu...,https://www.washingtonpost.com/world/2025/12/2...
48,31,2025-12-25-17-20-17 +0000,nypost,Zelensky says Ukrainians just want Putin dead ...,https://nypost.com/2025/12/25/world-news/zelen...
67,29,2025-12-25-15-51-43 +0000,nypost,Democrats warn Trump greenlighting Nvidia AI c...,https://nypost.com/2025/12/25/us-news/democrat...
128,28,2025-12-25-05-12-45 +0000,latimes,San Diego man killed by falling tree in third ...,https://www.latimes.com/california/story/2025-...
17,27,2025-12-25-19-28-21 +0000,nypost,New Yorkers shelled out more for Christmas gif...,https://nypost.com/2025/12/25/lifestyle/parent...
50,23,2025-12-25-17-10-52 +0000,nyt,"No Power, No Heat, No Water: Odesa’s Days of H...",https://www.nytimes.com/2025/12/25/world/europ...
61,22,2025-12-25-16-12-36 +0000,nypost,Bumbling crooks go to insane lengths to try to...,https://nypost.com/2025/12/25/us-news/video-sh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
